<a href="https://colab.research.google.com/github/sid8123/Matrix_decomposition_unitaries/blob/main/Decomposition1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane

     |████████████████████████████████| 409kB 7.7MB/s 
  Created wheel for pennylane: filename=PennyLane-0.14.1-cp37-none-any.whl size=481980 sha256=2c9b6a0d0f60341558deae3b05f54546bbd7d5b37ea04eba766a4c777757a6ba
  Stored in directory: /root/.cache/pip/wheels/6e/a9/c9/04941d6dd58b3c111cbd1389bf63ba1b23362c137359af4626
Successfully built pennylane


In [ ]:
from collections.abc import Iterable
import copy
import functools
from functools import reduce, lru_cache
from cachetools import cached, LRUCache, TTLCache
import inspect
import itertools
import numbers
from operator import matmul

import numpy as np

import pennylane as qml
from pennylane.variable import Variable

In [ ]:
def decompose(H, hide_identity=True):
    r"""Decomposes a Hermitian matrix into a linear combination of Pauli operators.

    Args:
        H (array[complex]): a Hermitian matrix of dimension :math:`2^n\times 2^n`
        hide_identity (bool): does not include the :class:`~.Identity` observable within
            the tensor products of the decomposition if ``True``

    Returns:
        tuple[list[float], list[~.Observable]]: a list of coefficients and a list
        of corresponding tensor products of Pauli observables that decompose the Hamiltonian.

    **Example:**

    We can use this function to compute the Pauli operator decomposition of an arbitrary Hermitian
    matrix:

    >>> A = np.array(
    ... [[-2, -2+1j, -2, -2], [-2-1j,  0,  0, -1], [-2,  0, -2, -1], [-2, -1, -1,  0]])
    >>> coeffs, obs_list = decompose_hamiltonian(A)
    >>> coeffs
    [-1.0, -1.5, -0.5, -1.0, -1.5, -1.0, -0.5, 1.0, -0.5, -0.5]

    We can use the output coefficients and tensor Pauli terms to construct a :class:`~.Hamiltonian`:

    >>> H = qml.Hamiltonian(coeffs, obs_list)
    >>> print(H)
    (-1.0) [I0 I1]
    + (-1.5) [X1]
    + (-0.5) [Y1]
    + (-1.0) [Z1]
    + (-1.5) [X0]
    + (-1.0) [X0 X1]
    + (-0.5) [X0 Z1]
    + (1.0) [Y0 Y1]
    + (-0.5) [Z0 X1]
    + (-0.5) [Z0 Y1]

    This Hamiltonian can then be used in defining VQE problems using :class:`~.ExpvalCost`.
    """
    n = int(np.log2(len(H)))
    N = 2 ** n

    if H.shape != (N, N):
        raise ValueError(
            "The Hamiltonian should have shape (2**n, 2**n), for any qubit number n>=1"
        )

    if not np.allclose(H, H.conj().T):
        raise ValueError("The Hamiltonian is not Hermitian")

    paulis = [qml.Identity, qml.PauliX, qml.PauliY, qml.PauliZ]
    obs = []
    coeffs = []

    for term in itertools.product(paulis, repeat=n):
        matrices = [i._matrix() for i in term]
        coeff = np.trace(functools.reduce(np.kron, matrices) @ H) / N
        coeff = np.real_if_close(coeff).item()

        if not np.allclose(coeff, 0):
            coeffs.append(coeff)

            if not all(t is qml.Identity for t in term) and hide_identity:
                obs.append(
                    functools.reduce(
                        matmul,
                        [t(i) for i, t in enumerate(term) if t is not qml.Identity],
                    )
                )
            else:
                obs.append(functools.reduce(matmul, [t(i) for i, t in enumerate(term)]))

    print(coeffs)
    print(obs)

In [ ]:
from datetime import datetime
start=datetime.now()

H = np.array([[ 1.5,  0,  0,  0.5, 0, 0.5, 0.5, 0],
       [ 0, 0.5,  0,  0,  0, 0, 0, 0.5],
       [ 0, 0, 0.5, 0, 0, 0, 0, 0.5],
       [0.5, 0, 0, -0.5, 0, 0, 0, 0],
       [0, 0, 0, 0, 0.5, 0, 0, 0.5],
       [0.5, 0, 0, 0, 0, -0.5, 0, 0],
       [0.5, 0, 0, 0, 0, 0, -0.5, 0],
       [0, 0.5, 0.5, 0, 0.5, 0, 0, -1.5]], dtype=np.complex128)
decompose(H)
print( datetime.now()-start)

[0.5, 0.25, -0.25, 0.5, 0.25, 0.25, -0.25, -0.25, 0.5]
[PauliZ(wires=[2]), PauliX(wires=[1]) @ PauliX(wires=[2]), PauliY(wires=[1]) @ PauliY(wires=[2]), PauliZ(wires=[1]), PauliX(wires=[0]) @ PauliX(wires=[2]), PauliX(wires=[0]) @ PauliX(wires=[1]), PauliY(wires=[0]) @ PauliY(wires=[2]), PauliY(wires=[0]) @ PauliY(wires=[1]), PauliZ(wires=[0])]
0:00:00.039305
